# Will it snow at the ESE 2024 Kongress?  

Copyright 2024 by [Doulos](https://www.doulos.com)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at:\
http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Part4- Quantization

### Read test data (needed for quantization)

In [1]:
import polars as pl
import os

path = "dataset"  # assume in dataset subfolder. Change as needed
x_test = pl.read_parquet(os.path.join(path,"x_test.parquet"))
y_test = pl.read_parquet(os.path.join(path,"y_test.parquet"))

### Helper function for quantization parameters

In [2]:
import tensorflow as tf

def representative_data_gen():
  data = tf.data.Dataset.from_tensor_slices(x_test)
  for i_value in data:
      i_value_f32 = tf.dtypes.cast(i_value, tf.float32)
      yield [i_value_f32]

### Convert regular TF model to TFLite quantized model

In [3]:
path="model"
converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(path,"snow_forecast"))
converter.representative_dataset = representative_data_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [4]:
tflite_model_quantized = converter.convert()

### Save the quantized model

In [5]:
tflite_quant_file = open("snow_model_quantized.tflite", "wb")
tflite_quant_file.write(tflite_model_quantized)
tflite_quant_file.close()

### Save model as a C bytes array
On Linux, we could simply use **xxd** for instance:
```
!xxd -i snow_model_quantized.tflite > model.cpp
```

On Windows, we could write a Python function.

In [6]:
def xxd_i(binfile, model_name="g_snow_detector", cpp_name="model.cpp"):
    with open(binfile,'rb') as f:
        data = f.read()

    header_name = cpp_name.replace(".cpp",".h")
    with open(cpp_name, 'w') as f:
        print(f'#include "{header_name}"\n', file=f)
        print(f"alignas(8) const unsigned char {model_name}[] = {{", file=f)

        # print the data in hex format grouping 12 bytes per line
        group_length = 12
        for i in range(0, len(data), group_length):
            line=', '.join(f'0x{byte:02x}' for byte in data[i:i+group_length])
            print(f'  {line},', file=f)
        
        print("};", file=f)
        print(f"const int {model_name}_len = {len(data)};", file=f)

In [7]:
xxd_i('snow_model_quantized.tflite')